In [8]:
class BottomPatternStrategy(BaseStrategy):
    """底分型策略 - 修复建仓条件和价格问题"""
    
    def __init__(self, data_provider, capital=100000, name="底分型策略"):
        super().__init__(data_provider, capital, name)
        
        # 策略状态
        self.a_line_high = None
        self.a_line_low = None
        self.a_line_date = None
        self.has_bought = False
        
        # 建仓时的A点信息（用于止损）
        self.entry_a_line_high = None  # 建仓时的A线最高点
        self.entry_a_line_low = None   # 建仓时的A线最低点（止损点）
        self.entry_a_line_date = None  # 建仓时的A线日期
        
     
        
        # A点记录（底分型策略特有）
        self.a_point_log = []
        
        logger.info(f"初始化底分型策略: {name}")
    
    def calculate_signals(self, stock_code, data, period=None):
        """计算底分型策略信号"""
        if len(data) < 2:
            return {}
        
        # 获取价格数据
        current_high = data.iloc[-1]['high']
        current_low = data.iloc[-1]['low']
        current_close = data.iloc[-1]['close']
        prev_low = data.iloc[-2]['low']
        prev_high = data.iloc[-2]['high']
        
        # 获取日期信息
        current_date = None
        if hasattr(data, 'index') and hasattr(data.index, 'dtype'):
            if pd.api.types.is_datetime64_any_dtype(data.index):
                current_date = data.index[-1]
            else:
                if 'datetime' in data.columns:
                    current_date = data.iloc[-1]['datetime']
                else:
                    current_date = datetime.now()
        else:
            current_date = datetime.now()
        
        # 获取实时价格
        current_price = self.data_provider.get_realtime_price(stock_code) or current_close
        
        # 如果已经建仓，只检查止损
        if self.has_bought:
            # 止损信号：买入后跌破建仓时的A线最低价
            stop_signal = (self.entry_a_line_low is not None and 
                          current_low < self.entry_a_line_low)

             # 即使在持仓状态下，也要检查是否形成新的A点（为后续交易做准备）
            a_line_condition = current_low < prev_low

             # 如果满足A线条件，更新A线信息（即使持仓中也要更新）
            if a_line_condition:
                self.a_line_high = current_high
                self.a_line_low = current_low
                self.a_line_date = current_date
                
                # 记录A点
                self.log_a_point(stock_code, current_high, current_low, current_date, "持仓中发现新的A点")
            
            signals = {
                'stop_signal': stop_signal,
                'current_price': current_price,
                'current_high': current_high,
                'current_low': current_low,
                'current_close': current_close,  # 添加收盘价
                'in_position': stock_code in self.positions,
                'has_bought': self.has_bought,
                'period': period,
                'current_date': current_date,
                'entry_a_line_low': self.entry_a_line_low,
                'prev_low': prev_low,
                'prev_high': prev_high
            }
            return signals
        
         # 如果未建仓，寻找A点和买入机会
        # 正常寻找A点
        a_line_condition = current_low < prev_low
        
        # 如果满足A线条件，更新A点信息
        if a_line_condition:
                self.a_line_high = current_high
                self.a_line_low = current_low
                self.a_line_date = current_date
                
                # 记录A点
                self.log_a_point(stock_code, current_high, current_low, current_date, "找到新的A点")
        
        # 买入信号：突破A线最高价且未买入过，且A点在止损之后
        buy_signal = (self.a_line_high is not None and 
                     current_high > self.a_line_high and 
                     not self.has_bought )
        
        # 构建信号字典
        signals = {
            'a_line_condition': a_line_condition,
            'a_line_high': self.a_line_high,
            'a_line_low': self.a_line_low,
            'a_line_date': self.a_line_date,
            'buy_signal': buy_signal,
            'stop_signal': False,  # 未建仓时没有止损信号
            'current_price': current_price,
            'current_high': current_high,
            'current_low': current_low,
            'current_close': current_close,  # 添加收盘价
            'in_position': stock_code in self.positions,
            'has_bought': self.has_bought,
            'period': period,
            'current_date': current_date,
            'prev_low': prev_low,
            'prev_high': prev_high
        }
        
        return signals
    
    def should_enter(self, stock_code, signals):
        """判断是否应该建仓"""
        # # 检查是否有有效的A点
        # if self.a_line_high is None or self.a_line_low is None:
        #     return False
            
        # # 检查A点是否在止损之后
        # if self.last_stop_date is not None and self.a_line_date <= self.last_stop_date:
        #     return False
            
        return signals.get('buy_signal', False) and not signals.get('in_position', False)
    
    def should_exit(self, stock_code, signals):
        """判断是否应该平仓（止损）"""
        # 只有在已建仓状态下才检查止损
        if not signals.get('has_bought', False):
            return False
            
        return signals.get('stop_signal', False) and stock_code in self.positions
    
    def should_add_position(self, stock_code, signals):
        """底分型策略不加仓"""
        return False
    
    def enter_position(self, stock_code, price, shares, reason="买入", trade_date=None):
        """建立仓位 - 使用建仓日收盘价"""
        # 获取信号中的日期信息
        signals = self.last_signals.get(stock_code, {})
        
        # 如果 trade_date 没有提供，从信号中获取
        if trade_date is None:
            trade_date = signals.get('current_date')
        
        # 使用建仓日收盘价作为建仓价格
        close_price = signals.get('current_close', price)
        
        success = super().enter_position(stock_code, close_price, shares, reason, trade_date)
        
        if success:
            self.has_bought = True
            
            # 保存建仓时的A点信息（用于止损）
            self.entry_a_line_high = self.a_line_high
            self.entry_a_line_low = self.a_line_low
            self.entry_a_line_date = self.a_line_date
            
            # 格式化A点日期显示
            if hasattr(self.entry_a_line_date, 'strftime'):
                a_line_date_str = self.entry_a_line_date.strftime('%Y-%m-%d %H:%M:%S')
            else:
                a_line_date_str = str(self.entry_a_line_date)
            
            logger.info(f"底分型买入成功: {stock_code}, "
                       f"建仓价格={close_price:.2f} (收盘价), "
                       f"A线最高={self.entry_a_line_high:.2f}, "
                       f"A线最低={self.entry_a_line_low:.2f} (止损点), "
                       f"A点日期={a_line_date_str}")
        
        return success
    
    def exit_position(self, stock_code, price, reason="止损", trade_date=None):
        """平仓 - 使用止损日收盘价"""
        # 获取信号中的日期信息
        signals = self.last_signals.get(stock_code, {})
        
        # 如果 trade_date 没有提供，从信号中获取
        if trade_date is None:
            trade_date = signals.get('current_date')
        
        # 使用止损日收盘价作为平仓价格
        close_price = signals.get('current_close', price)
        
        success = super().exit_position(stock_code, close_price, reason, trade_date)
        
        if success:
            # 记录止损信息
            logger.info(f"底分型止损成功: {stock_code}, "
                       f"止损价格={close_price:.2f} (收盘价), "
                       f"止损点={self.entry_a_line_low:.2f}")
            
                   
                     
            # 重置买入状态
            self.has_bought = False
            
             # 重置建仓时的A点信息，但保留当前的A线信息
            self.entry_a_line_high = None
            self.entry_a_line_low = None
            self.entry_a_line_date = None
                        
            
            # 记录止损后的状态
            if hasattr(trade_date, 'strftime'):
                stop_date_str = trade_date.strftime('%Y-%m-%d %H:%M:%S')
            else:
                stop_date_str = str(trade_date)
                
            logger.info(f"止损后重置状态，将寻找止损后的新A点，止损日期={stop_date_str}")
        
        return success
    
   
    
    def log_a_point(self, stock_code, a_line_high, a_line_low, date, reason="找到A点"):
        """记录A点信息（底分型策略特有）"""
       
            
        a_point_record = {
            'timestamp': date,
            'stock': stock_code,
            'a_line_high': a_line_high,
            'a_line_low': a_line_low,
            'reason': reason
        }
        self.a_point_log.append(a_point_record)
        
        # 格式化日期显示
        if hasattr(date, 'strftime'):
            date_str = date.strftime('%Y-%m-%d %H:%M:%S')
        else:
            date_str = str(date)
            
        logger.info(f"A点记录: {stock_code}, A线最高={a_line_high:.2f}, A线最低={a_line_low:.2f}, 日期={date_str}")